#🤝 **Predicción de Rotación de Empleados (Employee Churn Prediction)**

# **🧠Introducción al proyecto**

En el dinámico entorno empresarial actual, la rotación de empleados representa un desafío significativo que impacta directamente en los costos operativos, la productividad y la moral del equipo. Identificar a los empleados en riesgo de abandonar la empresa antes de que lo hagan permite a las organizaciones implementar intervenciones proactivas, reduciendo así las pérdidas asociadas y fomentando un ambiente de trabajo más estable y productivo.

Este proyecto aborda la problemática de la rotación mediante el desarrollo de un modelo predictivo de Machine Learning. Utilizando una combinación de datos estructurados (como información demográfica y laboral) y el análisis del feedback de los empleados a través de técnicas de Procesamiento de Lenguaje Natural (NLP), hemos construido una Red Neuronal Multimodal. Este enfoque nos permite no solo predecir qué empleados tienen una alta probabilidad de rotar, sino también entender los factores subyacentes que contribuyen a esta decisión.

# 🎯**Objetivos del Proyecto:**

El objetivo final es proporcionar a la dirección de Recursos Humanos una herramienta poderosa para optimizar la retención de talento y generar un impacto económico positivo tangible.

*   Predecir la Rotación de Empleados
*   Integrar Datos Textuales con NLP
*   Identificar Factores Clave de Rotación
*   Proporcionar Información Accionable a RRHH

# 🚀**Motivacion**

La motivación principal de este proyecto es dotar a las empresas de una herramienta basada en datos y potenciada por la inteligencia artificial que les permita no solo prever la fuga de talento, sino también comprender sus raíces.
Esto se traduce a:
*   Prever y comprender la fuga de talento
*   Reducir costos
*   Mejorar el clima laboral
* Optimizar la fuerza laboral





# **👥Audiencia**

Este proyecto esta dirigido a:


*   Líderes de Recursos Humanos (RRHH)
*   Gerencia General / Alta Dirección
*   Reclutadores / Talent Acquisition Specialists






# **💾Bases de datos**

https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset?resource=download

#📚**Librerias**

In [ ]:
# --- 1. Manejo de Datos y Visualización General ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots # Para subplots con Plotly
import random # Para la simulación de EmployeeFeedback

# --- 2. Preprocesamiento y Modelado de Machine Learning (Scikit-learn) ---
from sklearn.model_selection import train_test_split # Para dividir el dataset
from sklearn.preprocessing import StandardScaler, OneHotEncoder # Para escalar y codificar variables
from sklearn.compose import ColumnTransformer # Para aplicar transformaciones a columnas específicas
from sklearn.pipeline import Pipeline # Para encadenar pasos de preprocesamiento y modelo (aunque no lo usamos en la NN multimodal, es útil tenerlo)
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc # Métricas de evaluación

# --- 3. Procesamiento de Lenguaje Natural (NLP) ---
import nltk
import re # Para expresiones regulares en limpieza de texto
from nltk.corpus import stopwords # Para remover palabras comunes
from nltk.stem import WordNetLemmatizer # Para lematizar palabras (reducir a su forma base)
from nltk.sentiment.vader import SentimentIntensityAnalyzer # Para análisis de sentimiento VADER

# --- 4. Deep Learning (TensorFlow y Keras) ---
import tensorflow as tf
from tensorflow.keras.models import Model # Para construir modelos con múltiples entradas/salidas
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, GlobalMaxPooling1D, concatenate, Dropout # Capas de la red neuronal
from tensorflow.keras.optimizers import Adam # Optimizador para entrenar la red
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint # Callbacks para el entrenamiento

# --- 5. Interactividad (ipywidgets) ---
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, Layout # Widgets específicos
from IPython.display import display, clear_output # Para mostrar widgets y limpiar la salida

# --- Configuración Opcional para Visualización ---
sns.set_style("whitegrid") # Estilo de Seaborn
plt.rcParams['figure.figsize'] = (10, 7) # Tamaño de figura por defecto para Matplotlib

print("Todas las librerías necesarias han sido importadas exitosamente.")


Todas las librerías necesarias han sido importadas exitosamente.


# **📊Dataset y adicion de columna de comentarios simulados**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os


os.chdir('/content/drive/My Drive/Colab_Notebooks/data_3/PF_Bacigalupo/')

# Verificar que estás en el directorio correcto (opcional)
print("Directorio actual:", os.getcwd())

import pandas as pd


file_name = 'HR-Employee.csv'
df = pd.read_csv(file_name)

# Mostrar las primeras filas para verificar que se cargó correctamente
print("\nPrimeras 5 filas del dataset:")
print(df.head())

# Mostrar información general del dataset (tipos de datos, valores nulos)
print("\nInformación del dataset:")
df.info()

# Mostrar la distribución de la variable objetivo 'Attrition'
print("\nDistribución de la rotación (Attrition):")
print(df['Attrition'].value_counts())
print("\nPorcentaje de rotación:")
print(df['Attrition'].value_counts(normalize=True) * 100)

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Colab_Notebooks/data_3/PF_Bacigalupo/'

📜**Agregamos una columna de texto simulado**

In [ ]:

print("\nCreando columna 'EmployeeFeedback' con comentarios simulados en INGLÉS...")

# Frases de feedback negativo para empleados que rotan (en inglés)
negative_feedback = [
    "Lack of career growth, felt stagnant.",
    "Excessive workload, no recognition for overtime.",
    "Salary was not competitive compared to the market, affecting my motivation.",
    "Poor management and lack of supervisor support, leading to frustration.",
    "Work environment became toxic, with little team collaboration.",
    "No good work-life balance.",
    "Felt my ideas weren't valued, no room for innovation.",
    "Lack of adequate training and professional development.",
    "Issues with leadership, didn't feel heard or understood.",
    "Too much bureaucracy and inefficient processes."
]

# Frases de feedback positivo/neutral para empleados que no rotan (en inglés)
positive_feedback = [
    "Great work environment and a very collaborative team, I feel comfortable.",
    "Happy with the growth and development opportunities provided.",
    "Leadership is excellent, I feel supported and motivated by my supervisor.",
    "I really enjoy my role and the interesting challenges it presents daily.",
    "Benefits are competitive and there's a good work-life balance.",
    "I value the autonomy I have in performing my tasks.",
    "There are always new projects and the company cares about innovation.",
    "Performance reviews are fair and feedback is constructive.",
    "I feel part of the company's mission and my work has impact.",
    "Internal communication is clear and transparent."
]

# Función para asignar feedback basado en la columna 'Attrition'
def assign_feedback(attrition_status):
    if attrition_status == 'Yes':
        return np.random.choice(negative_feedback)
    else:
        return np.random.choice(positive_feedback)

# Aplicar la función para crear la nueva columna 'EmployeeFeedback'
df['EmployeeFeedback'] = df['Attrition'].apply(assign_feedback)

# Convertir 'Attrition' a numérica (0 o 1) en esta etapa para consistencia
# Si ya se hizo, no hay problema, simplemente se reasigna
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

print("\nColumna 'EmployeeFeedback' y 'Attrition' convertida a numérica añadida.")
print("Primeras 5 filas con las columnas 'Attrition' y 'EmployeeFeedback':")
print(df[['Attrition', 'EmployeeFeedback']].head())

print("\nEjemplos de empleados que rotaron (Attrition=1) y su feedback simulado:")
print(df[df['Attrition'] == 1][['Attrition', 'EmployeeFeedback']].head())

print("\nEjemplos de empleados que NO rotaron (Attrition=0) y su feedback simulado:")
print(df[df['Attrition'] == 0][['Attrition', 'EmployeeFeedback']].head())

print("\nInformación del dataset actualizada (para confirmar la nueva columna):")
df.info()

# **🔍Analisis y Preparacion del dataset**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6) # Ajusta el tamaño por defecto de los gráficos
plt.rcParams['font.size'] = 12 # Ajusta el tamaño de la fuente

**Vision general del Dataset**

In [ ]:
print("Información del dataset:")
df.info()

print("\nPrimeras 5 filas del dataset:")
print(df.head())

print("\nEstadísticas descriptivas de las columnas numéricas:")
print(df.describe())

**Análisis de la Variable Objetivo**

In [ ]:

df['Attrition_Label'] = df['Attrition'].map({0: 'No Rotación', 1: 'Sí Rotación'})

attrition_counts = df['Attrition_Label'].value_counts().reset_index()
attrition_counts.columns = ['Rotación', 'Cantidad de Empleados']

fig = px.pie(
    attrition_counts,
    values='Cantidad de Empleados',
    names='Rotación',
    title='Distribución de la Rotación de Empleados',
    hole=0.3, # Agrega un agujero para un gráfico de rosquilla
    color_discrete_sequence=px.colors.sequential.RdBu # Paleta de colores
)
fig.update_traces(textinfo='percent+label', pull=[0.05, 0]) # Muestra porcentaje y etiqueta, separa ligeramente 'Sí Rotación'
fig.show()

print("\nPorcentaje de Rotación:")
print(df['Attrition'].value_counts(normalize=True) * 100)

**Análisis de Características Numéricas Clave vs. Attrition**

In [ ]:
num_features_to_plot = ['Age', 'MonthlyIncome', 'DistanceFromHome', 'YearsAtCompany', 'HourlyRate', 'TotalWorkingYears']

for col in num_features_to_plot:
    fig = px.histogram(
        df,
        x=col,
        color='Attrition_Label', # Usa la etiqueta de rotación para diferenciar colores
        marginal='box', # Agrega boxplots en los márgenes para ver distribuciones
        nbins=50, # Número de "bins" para el histograma
        title=f'Distribución de {col} por Rotación',
        labels={'Attrition_Label': 'Rotación'},
        color_discrete_map={'No Rotación': 'blue', 'Sí Rotación': 'red'} # Colores específicos
    )
    fig.update_layout(bargap=0.1) # Espacio entre barras
    fig.show()

**Análisis Interactivo de Características Categóricas Clave vs. Attrition**

In [ ]:
cat_features_to_plot = ['Department', 'JobRole', 'EducationField', 'Gender', 'OverTime', 'BusinessTravel', 'MaritalStatus']

for col in cat_features_to_plot:
    # Calcular la tasa de rotación por cada categoría
    attrition_rate = df.groupby(col)['Attrition'].mean().reset_index()
    attrition_rate['Tasa de Rotación (%)'] = attrition_rate['Attrition'] * 100

    fig = px.bar(
        attrition_rate,
        x=col,
        y='Tasa de Rotación (%)',
        title=f'Tasa de Rotación por {col}',
        labels={'Tasa de Rotación (%)': 'Tasa de Rotación (%)'},
        color='Tasa de Rotación (%)', # Colorear por la tasa para un gradiente visual
        color_continuous_scale=px.colors.sequential.Plasma # Escala de color
    )
    fig.update_layout(xaxis_tickangle=-45) # Inclinar etiquetas para mejor lectura
    fig.show()

**Matriz de Correlación Interactiva (para variables numéricas)**

In [ ]:
# Calcula la correlación solo para columnas numéricas que no sean identificadores o constantes
# Y que 'Attrition' sea numérica (0 o 1)
df_numeric_for_corr = df.select_dtypes(include=np.number).drop(columns=['EmployeeNumber'], errors='ignore')
correlation_matrix = df_numeric_for_corr.corr()

fig = px.imshow(
    correlation_matrix,
    text_auto=True, # Muestra el valor de correlación en las celdas
    aspect="auto",
    color_continuous_scale=px.colors.sequential.RdBu, # Rojo-Azul para correlaciones positivas/negativas
    title='Matriz de Correlación de Variables Numéricas'
)
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False) # Eliminar cuadrícula para mayor claridad
fig.show()

print("\nCorrelación de características numéricas con Attrition (orden descendente):")
print(correlation_matrix['Attrition'].sort_values(ascending=False))

# 🗣️**NLP**

**LIMPIEZA Y NORMALIZACION BASICA DEL TEXTO**

In [ ]:

try:
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True) # Necesario para WordNetLemmatizer
    print("Recursos NLTK descargados exitosamente.")
except Exception as e:
    print(f"Error al descargar recursos NLTK: {e}.")
    print("Por favor, verifica tu conexión a internet o intenta descargar manualmente si persisten los problemas.")



stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Definir la función de limpieza y normalización básica
# Usamos text.split() para la tokenización simple por espacios en blanco,
# lo cual ayuda a evitar problemas con 'punkt_tab' de NLTK que a veces ocurren en ciertos entornos.
def limpieza_y_normalizacion_basica(text):
    if pd.isna(text):
        return ""
    text = str(text).lower() # Convertir a minúsculas y asegurar que sea string

    # 1. Remoción de Puntuación y Caracteres Especiales
    # Esto elimina todo lo que no sea una letra, número o espacio.
    text = re.sub(r'[^\w\s]', '', text)

    # 2. Tokenización (dividir el texto en palabras individuales)
    tokens = text.split()

    # 3. Remoción de Stopwords (eliminar palabras comunes que no aportan mucho significado)
    tokens = [word for word in tokens if word not in stop_words]

    # 4. Lematización (reducir las palabras a su forma base o raíz gramatical)
    # Por ejemplo, "running", "runs", "ran" se convertirían en "run".
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

print("Inicialización de librerías y función de limpieza básica completada.")



# --- Aplicar la función de limpieza al DataFrame ---
# Esto creará una nueva columna 'ProcessedFeedback' con el texto limpio.
df['ProcessedFeedback'] = df['EmployeeFeedback'].apply(limpieza_y_normalizacion_basica)



**COMPARACION TEXTO ORIGINAL VS. PREPROCESADO**

In [ ]:
print("Ejemplos de texto original vs. preprocesado (5 muestras aleatorias para comparar):")
print(df[['EmployeeFeedback', 'ProcessedFeedback']].sample(5, random_state=42))

print("\nLimpieza y normalización básica del texto completada. Columna 'ProcessedFeedback' creada y lista.")

# 😊**ANÁLISIS DE SENTIMIENTO Y POLARIDAD**

⬇️**Descarga del Léxico VADER e Inicializador de Sentimiento**

In [ ]:

try:
    nltk.download('vader_lexicon', quiet=True)
    print("Léxico VADER descargado exitosamente.")
except Exception as e:
    print(f"Error al descargar léxico VADER: {e}.")
    print("Por favor, verifica tu conexión a internet o intenta descargar manualmente.")

# Inicializar el analizador de sentimiento VADER
analyzer = SentimentIntensityAnalyzer()

➕**Función para obtener el score compuesto de sentimiento**

In [ ]:

def get_sentiment_score(text):
    if pd.isna(text) or text == "":
        return 0.0 # Devolver 0 para comentarios vacíos o NaN
    score = analyzer.polarity_scores(text)['compound']
    return score

 📊**Aplicación del Análisis de Sentimiento y Creación de la Columna**

In [ ]:
print("Aplicando análisis de sentimiento a 'EmployeeFeedback' y creando 'SentimentScore'...")

# Aplicar la función a la columna original 'EmployeeFeedback'
# Para VADER, el feedback original suele ser mejor porque conserva la puntuación y las exclamaciones que VADER sabe interpretar.
# Sin embargo, dado que ya lematizamos y limpiamos, usar 'ProcessedFeedback' también es válido
# si queremos que el score se base puramente en el contenido léxico.
# Por simplicidad y para aprovechar el procesamiento anterior, usaremos 'ProcessedFeedback'.
df['SentimentScore'] = df['ProcessedFeedback'].apply(get_sentiment_score)

print("Columna 'SentimentScore' añadida al DataFrame.")

👁️**Visualización de Ejemplos y Estadísticas del Sentimiento**

In [ ]:
# Mostrar ejemplos del feedback original, procesado y su puntuación de sentimiento
print("\nEjemplos de comentarios y sus puntuaciones de sentimiento (5 muestras aleatorias):")
print(df[['EmployeeFeedback', 'ProcessedFeedback', 'SentimentScore', 'Attrition']].sample(5, random_state=42))

# Ver la distribución del SentimentScore
print("\nDistribución del 'SentimentScore':")
print(df['SentimentScore'].describe())

📈**Visualización Gráfica de la Distribución del Sentimiento**

In [ ]:
import plotly.express as px
fig = px.histogram(df, x='SentimentScore', color='Attrition',
                   title='Distribución del Score de Sentimiento por Attrition',
                   labels={'SentimentScore': 'Puntuación de Sentimiento (VADER Compuesto)', 'Attrition': 'Rotación'},
                   nbins=50,
                   barmode='overlay',
                   opacity=0.7)
fig.show()

# 💪🧠**Preparación para Red Neuronal Convolucional/Recurrente y Consolidación de Datos**

En esta sección , daremos el paso crucial para preparar nuestro texto para la rama de la red neuronal convolucional (o recurrente).
* Vectorización de Texto para Keras
* Preparación de Características Estructuradas
* Definición de Entradas del Modelo (X_structured, tokenized_feedback, y)

📝**Preparación del Texto para la Rama de la Red Neuronal (TextVectorization) ----------**

In [ ]:

# Usamos el percentil 95 para capturar la mayoría de los comentarios sin hacer las secuencias excesivamente largas.
feedback_lengths = [len(str(x).split()) for x in df['ProcessedFeedback']]
max_sequence_length = int(np.percentile(feedback_lengths, 95))
if max_sequence_length == 0: # Asegurarse que no sea 0 si todos los comentarios son muy cortos
    max_sequence_length = 10 # Valor mínimo razonable si los comentarios son muy breves
print(f"Longitud máxima de secuencia (percentil 95 de tokens): {max_sequence_length}")


# Crear una capa TextVectorization de Keras
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=10000, # Un vocabulario de 10,000 palabras es un buen punto de partida.
    output_mode='int',
    output_sequence_length=max_sequence_length
)

# Adaptar la capa al texto de entrenamiento para construir el vocabulario.
print("Construyendo vocabulario y adaptando la capa TextVectorization a los comentarios procesados...")
text_data_for_vocab = np.array(df['ProcessedFeedback'].tolist()) # Convertir a NumPy array
vectorize_layer.adapt(text_data_for_vocab)

print(f"Vocabulario construido. Tamaño del vocabulario: {len(vectorize_layer.get_vocabulary())}")


# Transformar todos los comentarios (limpios) en secuencias numéricas usando la capa adaptada.
# Este 'tokenized_feedback' será una de las entradas a nuestra red neuronal híbrida.
tokenized_feedback = vectorize_layer(text_data_for_vocab).numpy()
print(f"\nComentarios transformados a secuencias numéricas (tokenized_feedback). Forma: {tokenized_feedback.shape}")
print("Primeras 5 secuencias numéricas de comentarios (primeros 10 tokens de cada una):")
print(tokenized_feedback[:5, :10])


🛠️**Preparación de Características Estructuradas y Definición de X, y ----------**

In [ ]:

# Columnas que no deben incluirse en las características estructuradas para el modelo.
# Esto incluye las columnas que ya hemos procesado para el texto, o identificadores.
cols_to_drop_from_structured = [
    'EmployeeCount',
    'StandardHours',
    'Over18',
    'EmployeeNumber',
    'EmployeeFeedback', # Texto original (ya manejado por la rama de texto)
    'ProcessedFeedback',# Texto limpio (ya convertido a tokenized_feedback)

]

# Crear el DataFrame para las características estructuradas
# Asegurarse de que 'Attrition' (la variable objetivo) se mantenga en este df temporalmente para la separación.
df_structured_features = df.drop(columns=cols_to_drop_from_structured, errors='ignore')

# Definir la variable objetivo (y)
# 'Attrition' ya debería ser 0/1 del paso de preparación del dataset.
y = df_structured_features['Attrition']

# Crear el conjunto de características estructuradas (X_structured)
# Eliminamos 'Attrition' de X_structured ya que es nuestra variable objetivo.
X_structured = df_structured_features.drop('Attrition', axis=1)

# Identificar las características numéricas y categóricas restantes en X_structured
# Estas listas serán usadas por ColumnTransformer de scikit-learn en el siguiente paso.
numeric_features = X_structured.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_structured.select_dtypes(include='object').columns.tolist()

print(f"\nCaracterísticas numéricas identificadas para la rama estructurada: {numeric_features[:10]}...") # Muestra solo las primeras 10
print(f"Características categóricas identificadas para la rama estructurada: {categorical_features}")

📈**Resumen de los Outputs Clave ----------**

In [ ]:
print("\n--- RESUMEN FINAL DEL PREPROCESAMIENTO DE DATOS PARA MODELO MULTIMODAL ---")
print(f"Variable objetivo 'y' (forma): {y.shape}")
print(f"Características estructuradas 'X_structured' (forma): {X_structured.shape}")
print(f"Características de texto procesadas 'tokenized_feedback' (forma): {tokenized_feedback.shape}")
print(f"Número de características numéricas para rama estructurada: {len(numeric_features)}")
print(f"Número de características categóricas para rama estructurada: {len(categorical_features)}")
print(f"Longitud de secuencia fija para texto: {max_sequence_length}")
print(f"Tamaño del vocabulario de texto: {len(vectorize_layer.get_vocabulary())}")

print("\nLa preparación avanzada del NLP y la consolidación de datos para el modelo multimodal han sido completadas.")
print("Los datos están ahora en el formato correcto para ser alimentados a una red neuronal convolucional/recurrente híbrida.")

#🧠 **Red Neuronal Multimodal**

Nuestra Red Neuronal Multimodal combina dos ramas: una CNN procesa el texto del feedback (incluyendo el sentimiento), mientras otra rama maneja los datos estructurados del empleado. Ambas se fusionan para aprender patrones complejos y predecir la rotación de manera más precisa, usando toda la información disponible.

**División de Datos y Preprocesamiento de Características Estructuradas**

Aquí realizaremos la división estándar de los datos y aplicaremos escalado y codificación a las características estructuradas, mientras que las secuencias de texto ya están preparadas (tokenized_feedback)

**División de Datos (Train/Test Split)**

In [ ]:

X_structured_train, X_structured_test, \
tokenized_feedback_train, tokenized_feedback_test, \
y_train, y_test = train_test_split(
    X_structured,
    tokenized_feedback,
    y,
    test_size=0.2,    # 20% para el conjunto de prueba
    random_state=42,  # Para reproducibilidad en la división
    stratify=y        # Mantiene la proporción de la clase 'Attrition' en ambos conjuntos
)

print(f"Forma de X_structured_train: {X_structured_train.shape}")
print(f"Forma de X_structured_test: {X_structured_test.shape}")
print(f"Forma de tokenized_feedback_train: {tokenized_feedback_train.shape}")
print(f"Forma de tokenized_feedback_test: {tokenized_feedback_test.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Forma de y_test: {y_test.shape}")
print("División de datos completada.")


**Aplicación de Escalado y Codificación**

In [ ]:
# Aplicamos StandardScaler a las características numéricas y OneHotEncoder a las categóricas.
# Las características de texto (tokenized_feedback) ya están en un formato numérico adecuado.
print("\nAplicando preprocesamiento (escalado numérico y One-Hot Encoding categórico) a las características estructuradas...")

**Creación de Transformadores**

In [ ]:
# Crear transformadores
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

**Configuración del ColumnTransformer**

In [ ]:
# Crear el ColumnTransformer
# Esto aplicará las transformaciones solo a las columnas relevantes de X_structured.
preprocessor_structured = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Mantiene columnas no especificadas si las hubiera
)

**Ajuste y Transformación de Datos de Entrenamiento**

In [ ]:
# Ajustar (fit) el preprocesador SOLO con los datos de entrenamiento estructurados
X_structured_train_processed = preprocessor_structured.fit_transform(X_structured_train)

**Transformación de Datos de Prueba**

In [ ]:
# Transformar (NO ajustar) los datos de prueba estructurados usando el preprocesador ya ajustado
X_structured_test_processed = preprocessor_structured.transform(X_structured_test)

**Confirmación de Dimensiones de los Datos Preprocesados**

In [ ]:
print(f"Forma de X_structured_train_processed después del preprocesamiento: {X_structured_train_processed.shape}")
print(f"Forma de X_structured_test_processed después del preprocesamiento: {X_structured_test_processed.shape}")
print("Preprocesamiento de características estructuradas completado.")

Resumen de Datos Listos para el Modelo
Ahora tenemos listos:
X_structured_train_processed, X_structured_test_processed (para la rama estructurada)
 tokenized_feedback_train, tokenized_feedback_test (para la rama de texto)
y_train, y_test (las etiquetas)

🏗️**Construcción de la Arquitectura del Modelo Multimodal (CNN/RNN Híbrida)**

En esta sección, definiremos la arquitectura de nuestra red neuronal. Será un modelo multimodal, lo que significa que tendrá dos entradas separadas: una para los datos estructurados y otra para las secuencias de texto. Ambas entradas se procesarán en ramas distintas antes de fusionarse para la predicción final.

In [ ]:
# Parámetros para la rama de texto
vocab_size = len(vectorize_layer.get_vocabulary())
embedding_dim = 128 # Dimensión del espacio de embedding para las palabras
filters = 128       # Número de filtros para la capa Conv1D
kernel_size = 5     # Tamaño del kernel (ventana) para la capa Conv1D

# Parámetros para la rama estructurada
structured_input_shape = X_structured_train_processed.shape[1]

# Parámetros generales del modelo
dense_units = 64    # Unidades en las capas densas ocultas
dropout_rate = 0.3  # Tasa de dropout para regularización
learning_rate = 0.001 # Tasa de aprendizaje para el optimizador Adam

print(f"\nParámetros del modelo:")
print(f"  Tamaño del vocabulario (texto): {vocab_size}")
print(f"  Dimensión de embedding: {embedding_dim}")
print(f"  Longitud de secuencia máxima: {max_sequence_length}")
print(f"  Número de características estructuradas: {structured_input_shape}")


**--- 1. Rama para Datos de Texto (CNN) ---**

In [ ]:


# ---  ---
print("\nConstruyendo la rama de la Red Neuronal Convolucional (CNN) para los datos de texto...")
text_input = Input(shape=(max_sequence_length,), name='text_input') # Entrada para las secuencias de texto tokenizadas
embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)(text_input)
conv_layer = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer) # Reducir la dimensionalidad de la salida de la CNN
text_branch_output = Dropout(dropout_rate)(pooling_layer)



**--- 2. Rama para Datos Estructurados ---**

In [ ]:

#
print("Construyendo la rama para los datos estructurados...")
structured_input = Input(shape=(structured_input_shape,), name='structured_input') # Entrada para los datos estructurados preprocesados
structured_dense_layer = Dense(dense_units, activation='relu')(structured_input)
structured_branch_output = Dropout(dropout_rate)(structured_dense_layer)



**--- 3. Fusión de las Ramas ---**

In [ ]:

#
print("Fusionando las salidas de ambas ramas...")
# Concatenar las salidas de ambas ramas para combinarlas
merged_output = concatenate([text_branch_output, structured_branch_output])

# Capas densas después de la fusión
combined_dense_1 = Dense(dense_units, activation='relu')(merged_output)
combined_dropout_1 = Dropout(dropout_rate)(combined_dense_1)
combined_dense_2 = Dense(dense_units // 2, activation='relu')(combined_dropout_1) # Otra capa densa más pequeña
combined_dropout_2 = Dropout(dropout_rate)(combined_dense_2)


**--- 4. Capa de Salida ---**

In [ ]:
#
# Para clasificación binaria (Attrition: Sí/No), usamos una capa Dense con 1 unidad y activación 'sigmoid'.
output_layer = Dense(1, activation='sigmoid', name='output_layer')(combined_dropout_2)


**--- 5. Creación y Compilación del Modelo ---**

In [ ]:
#
print("Creando y compilando el modelo multimodal...")
model = Model(inputs=[text_input, structured_input], outputs=output_layer)

# Usamos Adam como optimizador y binary_crossentropy para problemas de clasificación binaria.
# metrics: 'accuracy' para la exactitud, 'AUC' para el área bajo la curva ROC (buena para clases desbalanceadas).
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# Mostrar el resumen del modelo para ver la arquitectura
print("\nResumen de la arquitectura del modelo:")
model.summary()

print("\nArquitectura del modelo multimodal construida y compilada exitosamente.")

⚙️**Entrenamiento y Evaluación del Modelo Multimodal**

--- 1. Definición de Callbacks para el Entrenamiento ---

In [ ]:

print("\nConfigurando callbacks para el entrenamiento (EarlyStopping y ModelCheckpoint)...")

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_multimodal_model.keras', monitor='val_loss', save_best_only=True)

print("Callbacks configurados.")


--- 2. Entrenamiento del Modelo ---

In [ ]:

history = model.fit(
    x=[tokenized_feedback_train, X_structured_train_processed],
    y=y_train,
    epochs=50, # Puedes ajustar el número máximo de épocas
    batch_size=32, # Tamaño del batch, puedes experimentar con 16, 32, 64, etc.
    validation_data=([tokenized_feedback_test, X_structured_test_processed], y_test),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1 # Muestra el progreso del entrenamiento
)

print("\nEntrenamiento del modelo completado.")
print("El mejor modelo ha sido guardado como 'best_multimodal_model.keras'.")



 --- 3. Evaluación del Modelo en el Conjunto de Prueba ---

In [ ]:

best_model = tf.keras.models.load_model('best_multimodal_model.keras')

y_pred_proba = best_model.predict([tokenized_feedback_test, X_structured_test_processed]).ravel()
y_pred = (y_pred_proba > 0.5).astype(int)

# --- Métricas de Clasificación ---
print("\n--- Métricas de Clasificación ---")
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC Score: {auc_score:.4f}")

cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusión:")
print(cm)


--- 4. Visualización Interactiva de Métricas con Plotly Express ---

In [ ]:

# 4.1. Gráfico de Historial de Entrenamiento (Pérdida y Precisión)
print("Generando gráficos interactivos de pérdida y precisión del entrenamiento...")

hist_df = pd.DataFrame(history.history)
hist_df['epoch'] = hist_df.index + 1

fig_history = make_subplots(rows=1, cols=2, subplot_titles=('Pérdida del Modelo', 'Precisión del Modelo'))

# Gráfico de Pérdida
fig_history.add_trace(go.Scatter(x=hist_df['epoch'], y=hist_df['loss'], mode='lines', name='Pérdida de Entrenamiento'), row=1, col=1)
fig_history.add_trace(go.Scatter(x=hist_df['epoch'], y=hist_df['val_loss'], mode='lines', name='Pérdida de Validación'), row=1, col=1)
fig_history.update_xaxes(title_text='Época', row=1, col=1)
fig_history.update_yaxes(title_text='Pérdida', row=1, col=1)
fig_history.update_layout(hovermode="x unified") # Para mejor interactividad

# Gráfico de Precisión
fig_history.add_trace(go.Scatter(x=hist_df['epoch'], y=hist_df['accuracy'], mode='lines', name='Precisión de Entrenamiento'), row=1, col=2)
fig_history.add_trace(go.Scatter(x=hist_df['epoch'], y=hist_df['val_accuracy'], mode='lines', name='Precisión de Validación'), row=1, col=2)
fig_history.update_xaxes(title_text='Época', row=1, col=2)
fig_history.update_yaxes(title_text='Precisión', row=1, col=2)
fig_history.update_layout(title_text="Historial de Entrenamiento del Modelo", height=500, showlegend=True) # showlegend=True en layout principal
fig_history.show()


# 4.2. Curva ROC Interactiva
print("Generando curva ROC interactiva...")
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Crear DataFrame para la curva ROC para Plotly Express
roc_df = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'Threshold': thresholds})

fig_roc = px.area(
    roc_df,
    x="FPR", y="TPR",
    title=f'Curva ROC (Área = {roc_auc:.2f})',
    labels=dict(x='Tasa de Falsos Positivos (FPR)', y='Tasa de Verdaderos Positivos (TPR)'),
    width=700, height=500
)
fig_roc.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=0, y1=1
)
fig_roc.update_traces(hovertemplate="FPR: %{x:.2f}<br>TPR: %{y:.2f}<br>Threshold: %{customdata:.2f}") # Muestra threshold al pasar el mouse
fig_roc.data[0].customdata = roc_df['Threshold'] # Asigna los thresholds a customdata
fig_roc.show()


# 4.3. Matriz de Confusión Interactiva (Heatmap)
print("Generando matriz de confusión interactiva...")
labels = ['No Attrition', 'Attrition'] # Etiquetas para los ejes
cm_df = pd.DataFrame(cm, index=labels, columns=labels)

fig_cm = px.imshow(cm_df,
                   text_auto=True,
                   labels=dict(x="Predicción", y="Valor Real", color="Conteo"),
                   x=labels,
                   y=labels,
                   color_continuous_scale='Blues',
                   title='Matriz de Confusión',
                   width=600, height=500)
fig_cm.update_xaxes(side="top")
fig_cm.show()


print("\nEntrenamiento y evaluación del modelo multimodal completados con visualizaciones interactivas.")
print("El modelo está listo para su uso o para futuras optimizaciones.")

**Conclusión del Historial de Entrenamiento del Modelo**

La gráfica revela una convergencia extremadamente rápida y un excelente rendimiento, con pérdida mínima y precisión casi perfecta en entrenamiento y validación, indicando un modelo robusto y sin sobreajuste.

# 💰**Evaluacion economica**

En esta sección, calcularemos el impacto financiero de implementar nuestro modelo de predicción de rotación de empleados. Compararemos el costo anual de rotación sin el modelo versus el costo con el modelo en operación, considerando tanto los ahorros por retención como los costos de las intervenciones.

**--- 1. Definir Suposiciones Económicas Clave ---**

In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.graph_objects as go


y_pred_model = (y_pred_proba > 0.5).astype(int)
cm_opt = confusion_matrix(y_test, y_pred_model)



average_annual_salary = 60000 # Salario anual promedio de un empleado en USD
cost_factor_per_attrition = 1.5 # Costo total de rotación como un múltiplo del salario anual
num_employees_company = df.shape[0] # Número total de empleados en la empresa
attrition_rate_baseline = y_test.value_counts(normalize=True)[1] if 1 in y_test.value_counts(normalize=True).index else 0 # Tasa de rotación observada

cost_per_attrition = average_annual_salary * cost_factor_per_attrition
cost_per_false_positive_intervention = 500 # USD por intervención innecesaria
retention_success_rate = 0.30 # Tasa de éxito al retener empleados identificados por el modelo

print(f"Salario promedio: ${average_annual_salary:,.2f} | Costo por rotación (factor): {cost_factor_per_attrition}x")
print(f"Total empleados: {num_employees_company} | Tasa de rotación base: {attrition_rate_baseline:.2%}")
print(f"Costo por intervención FP: ${cost_per_false_positive_intervention:,.2f} | Éxito de retención: {retention_success_rate:.2%}")



**--- 2. Calcular el Impacto Económico del Modelo ---**

In [ ]:
# Desglose de la Matriz de Confusión del conjunto de prueba
TN = cm_opt[0, 0]
FP = cm_opt[0, 1]
FN = cm_opt[1, 0]
TP = cm_opt[1, 1]

# Calcular tasas de rendimiento del modelo
recall_model = TP / (TP + FN) if (TP + FN) > 0 else 0
false_positive_rate = FP / (TN + FP) if (TN + FP) > 0 else 0

# Costo Anual de Rotación SIN el Modelo (Escalado a la empresa)
actual_churners_company_annual = int(num_employees_company * attrition_rate_baseline)
total_cost_without_model = actual_churners_company_annual * cost_per_attrition

# Costo Anual de Rotación CON el Modelo (Escalado a la empresa)
churners_detected_annual = int(actual_churners_company_annual * recall_model)
churners_retained_annual = int(churners_detected_annual * retention_success_rate)

unnecessary_interventions_annual = int((num_employees_company - actual_churners_company_annual) * false_positive_rate)
cost_of_false_positives = unnecessary_interventions_annual * cost_per_false_positive_intervention

# Rotaciones que aún ocurrirán CON el modelo (no detectadas + detectadas pero no retenidas)
total_churners_with_model_intervention = (actual_churners_company_annual - churners_retained_annual)
cost_from_actual_churn_with_model = total_churners_with_model_intervention * cost_per_attrition

total_cost_with_model = cost_from_actual_churn_with_model + cost_of_false_positives

# Calcular el ahorro potencial neto anual del modelo
net_potential_savings = total_cost_without_model - total_cost_with_model


 --- 3. Crear la Tabla Interactiva con Plotly ---

In [ ]:
print("\n--- 3. Resumen Económico del Proyecto ---")

header_values = ["**Concepto**", "**Monto Estimado ($)**"]
cell_values = [
    ["Costo Anual de Rotación (SIN Modelo)",
     "Costo Anual de Rotación (CON Modelo)",
     "Ahorro Neto Anual Estimado del Modelo"],
    ['${:,.2f}'.format(total_cost_without_model),
     '${:,.2f}'.format(total_cost_with_model),
     '${:,.2f}'.format(net_potential_savings)]
]

cell_colors = [
    ['lightgray', 'lightgray', 'lightgray'],
    ['#FFCCCC', '#CCFFCC', '#CCCCFF']
]

fig = go.Figure(data=[go.Table(
    header=dict(values=header_values,
                fill_color='darkblue',
                align='left',
                font=dict(color='white', size=14)),
    cells=dict(values=cell_values,
               fill_color=cell_colors,
               align='left',
               font=dict(color='black', size=12)))
])

fig.update_layout(title_text="**Resumen Económico del Proyecto de Predicción de Rotación**", title_x=0.5)
fig.show()

print("\nEvaluación económica completada y presentada en una tabla interactiva.")